In [5]:
import requests

url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/InaFqKi-TlmTZwlzKvlNaQ/DV0101EN-Final-Assign-Part-2-Questions.py'
response = requests.get(url)

# Save the content to a local Python file
with open('DV0101EN-Final-Assign-Part-2-Questions.py', 'wb') as f:
    f.write(response.content)

# If you'd like to print the contents of the file
print(response.text)


#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import dash
import more_itertools
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

# Load the data using pandas
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# Set the title of the dashboard
#app.title = "Automobile Statistics Dashboard"

#---------------------------------------------------------------------------------
# Create the dropdown menu options
dropdown_options = [
    {'label': '...........', 'value': 'Yearly Statistics'},
    {'label': 'Recession Period Statistics', 'value': '.........'}
]
# List of years 
year_list = [i for i in range(1980, 2024, 1)]
#----------------------------------------------------------------------

In [12]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

# Load the data using pandas
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# Set the title of the dashboard
app.title = "Automobile Sales Statistics Dashboard"

# Create the dropdown menu options
dropdown_options = [
    {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
    {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'}
]

# List of years
year_list = [i for i in range(1980, 2024, 1)]

# Create the layout of the app
app.layout = html.Div([
    # Add title to the dashboard
    html.H1("Automobile Sales Dashboard", style={'textAlign': 'center'}),
    
    # Add two dropdown menus
    html.Div([
        html.Label("Select Report Type:"),
        dcc.Dropdown(
            id='report-type',
            options=dropdown_options,
            value='Yearly Statistics',
            placeholder='Select Report Type'
        )
    ], style={'width': '50%', 'margin': 'auto', 'padding': '20px'}),
    
    # Dropdown for selecting year
    html.Div([
        html.Label("Select Year:"),
        dcc.Dropdown(
            id='select-year',
            options=[{'label': str(i), 'value': i} for i in year_list],
            value=2020
        )
    ], style={'width': '50%', 'margin': 'auto', 'padding': '20px'}),
    
    # Output division to display graphs
    html.Div(id='output-container', style={'padding': '20px'})
])

# Callback function to update the dropdown
@app.callback(
    Output('select-year', 'disabled'),
    Input('report-type', 'value')
)
def update_input_container(selected_report):
    if selected_report == 'Recession Period Statistics':
        return False  # Enable the year dropdown for 'Recession Period Statistics'
    else:
        return True  # Disable the year dropdown for 'Yearly Statistics'

# Callback for plotting
@app.callback(
    Output('output-container', 'children'),
    [Input('report-type', 'value'), Input('select-year', 'value')]
)
def update_output(selected_report, input_year):
    if selected_report == 'Recession Period Statistics':
        # Filter the data for recession periods
        recession_data = data[data['Recession'] == 1]
        
        # Plot 1: Average automobile sales during the recession period
        yearly_rec = recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        chart1 = dcc.Graph(
            figure=px.line(yearly_rec, x='Year', y='Automobile_Sales', title="Average Automobile Sales during Recession Period")
        )
        
        # Plot 2: Average sales by vehicle type during the recession
        avg_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        chart2 = dcc.Graph(
            figure=px.bar(avg_sales, x='Vehicle_Type', y='Automobile_Sales', title="Average Sales by Vehicle Type during Recession")
        )
        
        # Plot 3: Advertising expenditure share by vehicle type during recessions
        exp_rec = recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        chart3 = dcc.Graph(
            figure=px.pie(exp_rec, names='Vehicle_Type', values='Advertising_Expenditure', title="Advertising Expenditure Share by Vehicle Type")
        )
        
        # Plot 4: Unemployment rate effect on vehicle sales
        unemp_data = recession_data.groupby(['unemployment_rate', 'Vehicle_Type'])['Automobile_Sales'].mean().reset_index()
        chart4 = dcc.Graph(
            figure=px.bar(unemp_data, x='unemployment_rate', y='Automobile_Sales', color='Vehicle_Type', title="Effect of Unemployment Rate on Vehicle Sales")
        )

        # Return the charts for recession period statistics
        return [
            html.Div([chart1, chart2], style={'display': 'flex'}),
            html.Div([chart3, chart4], style={'display': 'flex'})
        ]
    
    elif selected_report == 'Yearly Statistics':
        # Filter the data for the selected year
        yearly_data = data[data['Year'] == input_year]

        # Plot 1: Yearly automobile sales using a line chart
        yas = data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        chart1 = dcc.Graph(figure=px.line(yas, x='Year', y='Automobile_Sales', title='Yearly Automobile Sales'))

        # Plot 2: Total monthly automobile sales
        monthly_sales = data.groupby('Month')['Automobile_Sales'].sum().reset_index()
        chart2 = dcc.Graph(
            figure=px.line(monthly_sales, x='Month', y='Automobile_Sales', title='Total Monthly Automobile Sales')
        )

        # Plot 3: Average vehicles sold by vehicle type
        avg_sales_yearly = yearly_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()
        chart3 = dcc.Graph(
            figure=px.bar(avg_sales_yearly, x='Vehicle_Type', y='Automobile_Sales', title='Average Vehicles Sold by Vehicle Type')
        )

        # Plot 4: Total advertisement expenditure by vehicle type
        exp_data = yearly_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum().reset_index()
        chart4 = dcc.Graph(
            figure=px.pie(exp_data, names='Vehicle_Type', values='Advertising_Expenditure', title='Total Advertisement Expenditure by Vehicle Type')
        )

        # Return the charts for yearly statistics
        return [
            html.Div([chart1, chart2], style={'display': 'flex'}),
            html.Div([chart3, chart4], style={'display': 'flex'})
        ]
    else:
        return None

# Run the Dash app
if __name__ == '__main__':
    app.run(port=8052, debug=True)
